# Assignment

Jon Messier

2/27/23

 For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/jonme/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Minneapolis, MN'
TERM = 'brewery'

In [4]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location=LOCATION,
                                       term=TERM)
print(type(search_results))
print(search_results.keys())
search_results['total']

<class 'dict'>
dict_keys(['businesses', 'total', 'region'])


391

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_mpls_breweries.json"
JSON_FILE

'Data/results_in_progress_mpls_breweries.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_mpls_breweries.json already exists.


In [7]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:         
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
                  
    ## If it does NOT exist:
    else:    
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  


In [8]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_mpls_breweries.json already exists. Deleting previous file...
[i] Data/results_in_progress_mpls_breweries.json not found. Saving empty list to new file.
- 0 previous results found.


20

In [9]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,5drXZULD_FYpBDCfpFqgfQ,dangerous-man-brewing-co-minneapolis-2,Dangerous Man Brewing Co,https://s3-media2.fl.yelpcdn.com/bphoto/2VprEG...,False,https://www.yelp.com/biz/dangerous-man-brewing...,369,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 45.001118, 'longitude': -93.266388}",[],$$,"{'address1': '1300 2nd St NE', 'address2': '',...",+16122597283,(612) 259-7283,3433.459330
1,KAJtbn62rJJuyp5vcd3TXg,pryes-brewing-minneapolis-2,Pryes Brewing,https://s3-media2.fl.yelpcdn.com/bphoto/SXeeiT...,False,https://www.yelp.com/biz/pryes-brewing-minneap...,133,"[{'alias': 'breweries', 'title': 'Breweries'}]",4.5,"{'latitude': 44.99353, 'longitude': -93.27633}",[delivery],$$,"{'address1': '1401 W River Rd N', 'address2': ...",+16127877937,(612) 787-7937,2817.618545
2,nkLnhfd-OcIDQ28xHMKnJA,inbound-brewco-minneapolis,Inbound BrewCo,https://s3-media2.fl.yelpcdn.com/bphoto/jazO98...,False,https://www.yelp.com/biz/inbound-brewco-minnea...,129,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.5,"{'latitude': 44.9855537414551, 'longitude': -9...",[],$$,"{'address1': '701 N 5th St', 'address2': '', '...",+16122940375,(612) 294-0375,2300.658806
3,G7iXpVSRuXzCbD2daJ_JYQ,surly-brewing-minneapolis,Surly Brewing,https://s3-media3.fl.yelpcdn.com/bphoto/FDfXFH...,False,https://www.yelp.com/biz/surly-brewing-minneap...,1289,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.0,"{'latitude': 44.9732287, 'longitude': -93.2097...","[delivery, pickup]",$$,"{'address1': '520 Malcolm Ave SE', 'address2':...",+17635353330,(763) 535-3330,4086.609486
4,b1PSsTkD2sV3DhitJHHkPw,lakes-and-legends-brewing-minneapolis-3,Lakes & Legends Brewing,https://s3-media2.fl.yelpcdn.com/bphoto/RwWJJs...,False,https://www.yelp.com/biz/lakes-and-legends-bre...,162,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.5,"{'latitude': 44.9689411, 'longitude': -93.2795...",[],$$,"{'address1': '1368 Lasalle Ave', 'address2': N...",+16129996020,(612) 999-6020,1425.516649


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
363,xwiykwyjkbVSIw5xDK6S-g,liberty-village-wines-and-spirits-stillwater,Liberty Village Wines & Spirits,https://s3-media3.fl.yelpcdn.com/bphoto/bUsWAB...,False,https://www.yelp.com/biz/liberty-village-wines...,12,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",2.0,"{'latitude': 45.0561637878418, 'longitude': -9...",[],$$,"{'address1': '105 New England Pl', 'address2':...",+16514393022,(651) 439-3022,32808.372278
364,dQLZ1yutkYdVT80w9T99nw,ngon-vietnamese-bistro-saint-paul-2,Ngon Vietnamese Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/5dVwkV...,False,https://www.yelp.com/biz/ngon-vietnamese-bistr...,320,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"{'latitude': 44.9559741278075, 'longitude': -9...",[delivery],$$,"{'address1': '799 University Ave', 'address2':...",+16512223301,(651) 222-3301,10178.823433
365,Rvf8Vw4ALQey_ZALiAp_MA,crave-american-kitchen-and-sushi-bar-edina-2,CRAVE American Kitchen & Sushi Bar,https://s3-media1.fl.yelpcdn.com/bphoto/h1Zctt...,False,https://www.yelp.com/biz/crave-american-kitche...,268,"[{'alias': 'newamerican', 'title': 'American (...",3.0,"{'latitude': 44.8769816139583, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '3520 West 70th St', 'address2': ...",+19526976000,(952) 697-6000,11596.350623
366,ZKcegrvDf2ZRkLoy-0gH4A,mgm-liquor-warehouse-chanhassen,Mgm Liquor Warehouse,,False,https://www.yelp.com/biz/mgm-liquor-warehouse-...,7,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",2.5,"{'latitude': 44.8611114, 'longitude': -93.5382...",[],$$$,"{'address1': '7856 Market Blvd', 'address2': '...",+19529379300,(952) 937-9300,25075.603764
367,ZLV4UPkFdkDT15Ih5s09Vw,los-ocampo-saint-paul,Los Ocampo,https://s3-media4.fl.yelpcdn.com/bphoto/3iG7wL...,False,https://www.yelp.com/biz/los-ocampo-saint-paul...,77,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.0,"{'latitude': 44.95602, 'longitude': -93.126}","[pickup, delivery]",$$,"{'address1': '615 University Ave W', 'address2...",+16513405311,(651) 340-5311,10802.073409


In [11]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [13]:
#save the final results to a compressed csv
final_df.to_csv('Data/final_results_mpls_breweries.csv.gz', compression='gzip',index=False)